# Resumen del Proceso de Extracción de IDs para Datos de AcousticBrainz

En este notebook, documentamos el proceso para extraer datos de la API de MusicBrainz y enriquecer nuestro dataset con `recording_id` de las canciones para después sacar todas los valores porcentuales de las características de cada canción. A continuación, se describe el flujo de trabajo:

1. **📚 Importación de Librerías**:
    - Importamos las librerías necesarias como `numpy`, `pandas`, `requests`, `time` y `os`.

2. **📂 Carga y Preparación del Dataset**:
    - Cargamos el dataset inicial desde un archivo CSV.
    - Filtramos las columnas necesarias (`song_name` y `artist_name`) y guardamos un archivo simplificado.

3. **🔧 Definición de Funciones**:
    - Definimos la función `get_recording_id(df)` que se encarga de obtener el `recording_id` de cada canción utilizando la API de MusicBrainz.
    - La función maneja solicitudes HTTP, procesa las respuestas y guarda los resultados en archivos temporales (`nuevos_ids_encontrados.csv.temp` y `ids_no_encontrados.csv.temp`).

4. **🗂️ Inicialización del Dataset**:
    - Cargamos el dataset a procesar y nos aseguramos de que la columna `recording_id` esté presente.

5. **🚀 Ejecución del Proceso de Extracción**:
    - Ejecutamos la función `get_recording_id(df)` para obtener los `recording_id` y guardarlos en los archivos correspondientes.

6. **📊 Resultados**:
    - Los resultados se guardan en dos archivos:
      - `nuevos_ids_encontrados.csv.temp`: Contiene las canciones para las cuales se encontró un `recording_id`.
      - `ids_no_encontrados.csv.temp`: Contiene las canciones para las cuales no se encontró un `recording_id`.

7. **🔍 Consideraciones Adicionales**:
    - Si se vuelve a ejecutar la función, los nuevos datos se añaden al final de los archivos existentes para evitar duplicados.
    - Se recomienda eliminar los datos ya procesados de los archivos temporales antes de una nueva ejecución para optimizar el tiempo de procesamiento.

Este flujo de trabajo nos permite enriquecer nuestro dataset con información adicional de MusicBrainz de manera eficiente y organizada.


In [ ]:
# Importación de Librerías
import numpy as np
import pandas as pd
import requests # para realizar solicitudes HTTP y obtener datos de una API externa, en este caso Brainz
import time
import os # para interactuar con el sistema de archivos, permitiendo la lectura y escritura de archivos necesarios para el análisis

##### ELEGIR LA PARTE DEL DATA SET CON EL QUE EMPEZAMOS 
  
Nos hemos dividido los datos para ir sacándolos poco a poco ya que el dataset es muy grande y solo hemos podido procesar una pequeña parte de él.  
A futuro se puede ver como podemos seguir extrayendo datos

In [ ]:
import pandas as pd
import os

# Rutas de los archivos
final_output_path = "../data/split_datasets/genius_songs_batch_1.csv"
simple_output_path = "../data/raw/1_toprocess.csv"

# Cargar el archivo final existente
if os.path.exists(final_output_path):
    final_df = pd.read_csv(final_output_path)
    # Filtrar solo las columnas necesarias
    simple_df = final_df[['song_name', 'artist_name']]
    
    # Guardar el resultado en un nuevo archivo
    simple_df.to_csv(simple_output_path, index=False)
    print(f"Archivo simplificado guardado en: {simple_output_path}")
    print(simple_df.head(), simple_df.shape)
else:
    print(f"El archivo final {final_output_path} no existe. Por favor, verifica la ruta.")


Archivo simplificado guardado en: C:\Users\solan\Downloads\get_data_from_songs\data\raw\1_toprocess.csv
                     song_name                     artist_name
0               gasolina remix                    daddy yankee
1                        rompe                    daddy yankee
2  we no speak americano remix                         pitbull
3           suenos en realidad                        ozomatli
4              triste me pongo  califas (mr. lil one & shisty) (10000, 2)


# Ejemplo de uso de la función para obtener nuevos recording_ids

`df = pd.read_csv(ruta_al_dataset_a_procesar.csv)`

`get_recording_id(df)`

# Función para obtener nuevos recording_ids

In [11]:
def get_recording_id(df):
    # Función para obtener el recording_id desde la API de MusicBrainz
    def get_recording_id(title, artist):
        query = f'recording:"{title}" AND artist:"{artist}"'
        url = f"https://musicbrainz.org/ws/2/recording/?query={query}&fmt=json"

        try:
            response = requests.get(url, timeout=10)  # Agregar timeout para evitar bloqueos indefinidos
            if response.status_code == 200:
                data = response.json()
                if "recordings" in data and len(data["recordings"]) > 0:
                    return data["recordings"][0]["id"]
            return None
        except requests.exceptions.RequestException as e:
            print(f"Error on search '{title}' - '{artist}': {e}")
            return None

    # Rutas de los archivos de salida
    found_file = r"C:\Users\solan\Downloads\get_data_from_songs\data\raw\1_nuevos_ids_encontrados.csv.temp"
    not_found_file = r"C:\Users\solan\Downloads\get_data_from_songs\data\1ids_no_encontrados.csv.temp"

    # Crear los archivos si no existen
    # En caso de que ya existan y tengan registros, en lugar de sobrescribirse, los nuevos se añaden al final
    if not os.path.exists(found_file):
        pd.DataFrame(columns=["artist_name", "song_name", "recording_id"]).to_csv(found_file, index=False)
    if not os.path.exists(not_found_file):
        pd.DataFrame(columns=["artist_name", "song_name"]).to_csv(not_found_file, index=False)

    # Inicializar contadores
    total_requests = 0

    # Procesar el DataFrame
    for index in df.index:
        row = df.loc[index]

        # Saltar filas con un recording_id ya existente y válido
        if pd.notna(row["recording_id"]) and row["recording_id"] != "Not found":
            continue

        total_requests += 1
        print(f"\n[Progress] Request {total_requests}/{len(df)}...")

        # Obtener el recording_id
        recording_id = get_recording_id(row["song_name"], row["artist_name"])

        # Procesar y guardar los resultados por cada request
        # Es un poco más lento pero resulta en menos fallos, por lo que es preferible
        if not recording_id:  # Si no se encuentra el recording_id, se almacenan los datos procesados en un dataset ids_no_encontrados.csv.temp
            print(f"  ⚠️ Not Recording ID found for: {row['song_name']} - {row['artist_name']}")
            not_found_df = pd.DataFrame([{"artist_name": row["artist_name"], "song_name": row["song_name"]}])
            not_found_df.to_csv(not_found_file, mode="a", index=False, header=False)
        else:  # Si se encuentra el recording_id, se almacenan los datos procesados en un dataset nuevos_ids_encontrados.csv.temp
            print(f"  ✅ Recording ID found: {recording_id}")
            found_df = pd.DataFrame([{
                "artist_name": row["artist_name"],
                "song_name": row["song_name"],
                "recording_id": recording_id
            }])
            found_df.to_csv(found_file, mode="a", index=False, header=False)


        # Respetar el límite de velocidad de la API
        time.sleep(1)

    print(f"\nProcessing complete. Results saved in:\n - Found: {found_file}\n - Not Found: {not_found_file}")

# Obtener nuevos recording_id

In [12]:
df = pd.read_csv(r'C:\Users\solan\Downloads\get_data_from_songs\data\raw\1_toprocess.csv')

# Inicializar la columna 'recording_id' si no existe
if 'recording_id' not in df.columns:
    df['recording_id'] = None  # O un valor predeterminado como "Not found"

In [13]:
get_recording_id(df)


[Progress] Request 1/10000...
  ✅ Recording ID found: 07e4fa19-44db-415b-bd1a-32d72419b819

[Progress] Request 2/10000...
  ✅ Recording ID found: 156da9c1-0ccf-4205-b5c5-92d01470867f

[Progress] Request 3/10000...
  ⚠️ Not Recording ID found for: we no speak americano remix - pitbull

[Progress] Request 4/10000...
  ✅ Recording ID found: 74475a5e-6773-4dc2-91ac-3cc16793ab03

[Progress] Request 5/10000...
  ✅ Recording ID found: ef3d1f71-d2fc-4ea4-b69c-d3f6a6d148b7

[Progress] Request 6/10000...
  ✅ Recording ID found: 72b7c164-91f7-4f1c-aaa3-b104d2e86f9b

[Progress] Request 7/10000...
  ⚠️ Not Recording ID found for: quieren pleito - crooked stilo

[Progress] Request 8/10000...
  ✅ Recording ID found: 3227a55b-ae2a-4a05-b602-30c6980ce1d1

[Progress] Request 9/10000...
  ✅ Recording ID found: a9bb0335-f3f3-41c8-9bfc-e4458016a2ca

[Progress] Request 10/10000...
  ✅ Recording ID found: 2d18355a-c382-4579-bccc-fb5ebaa3128d

[Progress] Request 11/10000...
  ✅ Recording ID found: 721ce512-4

Una vez termine o se corte la ejecución, los datos de los que se ha encontrado recording_id estarán en [nuevos_ids_encontrados.csv.temp](../data/raw/nuevos_ids_encontrados.csv.temp) y los que no se encuentren, en [ids_no_encontrados.csv.temp](../data/raw/ids_no_encontrados.csv.temp).

En caso de que se vuelva a ejecutar la función, si los archivos ya existen y contienen datos, estos no se sobreescriben, sino que se añaden al final. Para evitar duplicados y ahorrar tiempo, lo ideal es eliminar los datos que ya están en [nuevos_ids_encontrados.csv.temp](../data/raw/nuevos_ids_encontrados.csv.temp) y [ids_no_encontrados.csv.temp](../data/raw/ids_no_encontrados.csv.temp) del dataset a procesar. Para ello tenemos la función [remove_existing_records()](eliminar-de-un-dataset-que-existen-en-otro.ipynb)